In [ ]:
import ee
from google.colab import drive
#drive.mount('data')
drive.mount('/content/drive')
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='abharajan')

import sys
sys.path.append('/content/drive/My Drive/BiomassApril2024')

import geemap

import numpy as np
import pandas as pd

import utilceda as uc
import coordinates as cdn

Mounted at /content/drive


In [ ]:
# Choose coordinates to create geometries for datasets

coordinatesList = [[38.99458221563848, 6.463386197712386]]
placeList = ['Africa' ]

cList = [cdn.clIndia, cdn.clSEAsia, cdn.clAfrica, cdn.clSA, cdn.clNA, cdn.clEu]
area = ['India', 'SouthEastAsia', 'Africa', 'SouthAmerica', 'NorthAmerica', 'Europe']

iArea = 0
# coordinatesList = cList[iArea]

# Folder where the EDA and model analysis will be saved
#folder = 'drive/MyDrive/BiomassApril2024/EDAModelData/' + area[iArea] + '/'
folder = 'BiomassApril2024'

# no of coordinates
ncoor = len(coordinatesList)

# placeList =[]
# for icoor in range(0, ncoor):
#   place = area[iArea] + icoor.astype(str) +'_' + str(round(coordinatesList[icoor][0],2)) + '_' +  str(round(coordinatesList[icoor][1],2))
#   print(place)
#   placeList.append(place)


# Select the satellite image and biomass data sources
# ****************************************************

# ESA CCI estimates for biomass
biomass_cci = ee.ImageCollection("projects/sat-io/open-datasets/ESA/ESA_CCI_AGB")

# Sentinel-2 Surface Reflectance
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

# Select a time-period
# ****************************************************
startDate = ee.Date.fromYMD(2019, 1, 1)
endDate = startDate.advance(1, 'year')


# Choose the grid size and projection
gridScale = 100
generalProj = ee.Projection('EPSG:3857').atScale(gridScale)

# no of coordinates
ncoor = len(coordinatesList)

projNameAr = ['S2', 'CCI', 'general']

is2=0
icci=1
iall=2

s2ProjName = projNameAr[is2]
cciProjName = projNameAr[icci]
allProjName = projNameAr[iall]

projectionType = 'S2' + s2ProjName + 'CCI' + cciProjName + 'All' + allProjName

# Create dataframes
for i in range(0, ncoor):

  # Geometry
  geometry = uc.makeGeometry(coordinatesList[i])

  # setting projections
  s2Proj = uc.getS2projection(s2, startDate, endDate, geometry)
  cciProj = uc.getCCIprojection(biomass_cci, startDate, endDate, geometry)

  projAr = [s2Proj, cciProj, generalProj]

  gridProjectionCCI = projAr[icci]
  gridProjectionS2 = projAr[is2]
  gridProjectionAll = projAr[iall]


  # Obtaining processed satellite images and biomass datasets
  s2Composite = uc.s2CompositePrep(s2, startDate, endDate, gridProjectionS2, geometry)
  cciMosaic = uc.cciPrep(biomass_cci, startDate, endDate, gridProjectionCCI, geometry)

  # Create stack of images
  ar = [s2Composite, cciMosaic] # array of the images to be stacked
  nimages = len(ar)   # number of images to be stacked
  stackedCEDA = uc.stack(ar, nimages, gridProjectionAll)


  # Convert the stack to a data file
  filename = placeList[i]+'_' + projectionType
  uc.makeCSVCEDA(stackedCEDA, geometry, gridScale, folder, filename)


38.99458221563848 6.463386197712386 39.32058221563848 6.679066197712386
